# **REGex Final Project TEAM 10**

**Contributors:**
* Yashraj Bharambe
* Diptesh Chaudhari
* Pranav Karanjawane
* Shrikrushnakumar Sondge
* Ashutosh Mishra

**Project Description:**
A computer vision approach to classifying garbage into recycling categories could be an efficient way to process waste. Recycling is already significant work for all countries. Among the work needed for recycling, garbage classification is the most fundamental step to enable cost-efficient recycling. This project aims to take images of a single piece of recycling or garbage and classify it into one out of six classes consisting of glass, paper, metal, plastic, cardboard, and trash.

# **Importing Libraries**

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Setting up File locations and Checking sample images**

In [14]:
# setting the path and the labels list for classification of targets on the basis in human understandable form

train_dir = os.path.join('/kaggle/input/garbage-classification/TRAIN')
labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [15]:
# checking the size of data available to us for training out model

for label in labels:
    directory = os.path.join(train_dir, label)
    print("Images of label \"" + label + "\":\t", len(os.listdir(directory)))

In [16]:
# plotting images of different review for understanding the dataset

plt.figure(figsize=(30,14))

for i in range(6):
    directory = os.path.join(train_dir, labels[i])
    for j in range(10):
        path = os.path.join(directory, os.listdir(directory)[j])
        img = mpimg.imread(path)
        
        plt.subplot(6, 10, i*10 + j + 1)
        plt.imshow(img)
        
        if j == 0:
            plt.ylabel(labels[i], fontsize=20)
        
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);
plt.tight_layout()
plt.show()

In [17]:
# checking size of individual image

directory = os.path.join(train_dir, 'cardboard')
path = os.path.join(directory, os.listdir(directory)[0])
image = mpimg.imread(path)
image.shape

# **Deep Learning**

In [18]:
# creating the model

model = tf.keras.models.load_model("../input/lastone/lastone.h5")

model.summary()

In [19]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = 0.0002), metrics=['accuracy'])

In [20]:
# creating generators for combining data and increasing the gainable insights by slightly modifying the images in the dataset

train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,
                                   rotation_range=15,zoom_range=0.1,
                                   width_shift_range=0.15,height_shift_range=0.15,
                                   shear_range=0.1,
                                   fill_mode="nearest",
                                   rescale=1./255., 
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 200), batch_size=32, class_mode = 'binary', subset='training')
validation_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 200), batch_size=32, shuffle = True, class_mode = 'binary', subset='validation')

In [21]:
# creating the callback function so that it can be used to end the training in case reached a good accuracy rate (above 90%)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [22]:
history = model.fit(train_generator, epochs=30, verbose=1, validation_data=validation_generator, callbacks=[callbacks])

In [23]:
model.save("lastone.h5")

In [24]:
from PIL import Image
#cat = int(input('Enter any category by index: '))
#ind = int(input('Enter any index to test: '))
for cat in range(6):
    directory = os.path.join(train_dir, labels[cat % 6])
    for ind in range(10, 15):
        path = os.path.join(directory, os.listdir(directory)[ind])
        img = Image.open(path)
        img = img.resize((150, 200))
        display(img)
        x = keras.preprocessing.image.img_to_array(img)
        x = x/255
        x = np.expand_dims(x, axis=0)

        images = np.vstack([x])
        classes = model.predict(images)
        pred = labels[np.argmax(classes)]
    
        print("Actual: ", labels[cat % 6], " Prediction: ", pred)